# Notebook 03: Développement et Test des Agents Individuels et de leurs Outils

Ce notebook se concentre sur le test et la démonstration de chaque agent que nous avons défini dans `src/agents/agent_architectures.py`. Nous allons instancier chaque agent, lui soumettre des tâches spécifiques, et observer comment il utilise ses outils (définis dans `src/agents/tool_definitions.py`) et comment il génère ses réponses.

**Prérequis :**
* Avoir exécuté `00_setup_environment.ipynb` (environnement configuré, clés API dans `.env`).
* Pour tester `DocumentAnalysisAgent` efficacement, il est préférable d'avoir une base de données MongoDB peuplée via `01_data_ingestion_and_embedding.ipynb` (ou `scripts/run_ingestion.py`) et que le `RetrievalEngine` (utilisé par `knowledge_base_retrieval_tool`) soit fonctionnel.

In [ ]:
import logging
import sys
from pathlib import Path
import os
import json # Pour un affichage lisible des sorties d'outils

# Ajout de la racine du projet au PYTHONPATH
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

from dotenv import load_dotenv
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}.")

from config.settings import settings
from config.logging_config import setup_logging

# Importer les fonctions de création d'agents
from src.agents.agent_architectures import (
    create_research_planner_agent,
    create_arxiv_search_agent,
    create_document_analysis_agent,
    create_synthesis_agent,
    get_llm 
)

# Importer les outils pour d'éventuels tests directs
from src.agents.tool_definitions import arxiv_search_tool, knowledge_base_retrieval_tool
# document_deep_dive_analysis_tool est aussi disponible mais moins susceptible d'être testé directement ici

# Importer les types de messages LangChain
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

setup_logging(level="INFO") # Mettre à DEBUG pour voir les étapes internes des agents
logger = logging.getLogger("nb_03_agent_testing")

# --- MODIFIÉ : Vérification des prérequis pour les LLMs et Embeddings ---
# Pour les agents génératifs (Planner, ArxivSearcher, DocAnalyzer, Synthesizer, CrewAI via get_llm)
active_llm_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"Les agents utiliseront le fournisseur LLM génératif : '{active_llm_provider}'")
if active_llm_provider == "openai" and not settings.OPENAI_API_KEY:
    logger.error(f"ERREUR : Le fournisseur LLM est 'openai', mais OPENAI_API_KEY n'est pas configurée. Les tests des agents utilisant ce LLM échoueront.")
elif active_llm_provider == "huggingface_api" and not settings.HUGGINGFACE_API_KEY:
    logger.error(f"ERREUR : Le fournisseur LLM est 'huggingface_api', mais HUGGINGFACE_API_KEY n'est pas configurée.")
elif active_llm_provider == "ollama" and not settings.OLLAMA_BASE_URL:
    logger.error(f"ERREUR : Le fournisseur LLM est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")

# Pour knowledge_base_retrieval_tool (qui utilise RetrievalEngine, qui utilise le provider d'embedding configuré)
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Le 'knowledge_base_retrieval_tool' utilisera le fournisseur d'embedding : '{active_embedding_provider}'")
if active_embedding_provider == "openai" and not settings.OPENAI_API_KEY:
    logger.error(f"ERREUR : Le fournisseur d'embedding pour RetrievalEngine est 'openai', mais OPENAI_API_KEY n'est pas configurée. Le 'knowledge_base_retrieval_tool' échouera probablement.")
elif active_embedding_provider == "ollama" and not settings.OLLAMA_BASE_URL:
     logger.error(f"ERREUR : Le fournisseur d'embedding pour RetrievalEngine est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")
# Pas de vérification de clé pour HuggingFace embeddings locaux.

if not settings.MONGO_URI: # Nécessaire pour knowledge_base_retrieval_tool via RetrievalEngine
    logger.error("ERREUR : MONGO_URI non trouvé. Le 'knowledge_base_retrieval_tool' ne pourra pas se connecter à la base de données.")

if not settings.TAVILY_API_KEY: 
    logger.warning("Clé API TAVILY non configurée (non utilisée par les outils actuels de ce notebook, mais bon à savoir).")
# --- FIN MODIFIÉ ---

### 1. Test du `ResearchPlannerAgent`

Cet agent est conçu pour prendre une requête utilisateur complexe et la décomposer en un plan de recherche structuré. Il n'utilise pas d'outils pour cette tâche, se basant uniquement sur ses instructions (prompt système) et le LLM.

In [ ]:
if settings.OPENAI_API_KEY:
    logger.info("--- Test du ResearchPlannerAgent ---")
    planner_agent_executor = create_research_planner_agent()
    
    # Requête utilisateur d'exemple
    user_query_plan = "What are the latest trends and key challenges in applying deep reinforcement learning to multi-robot navigation and coordination, particularly for swarm robotics?"
    logger.info(f"Requête pour le planificateur : '{user_query_plan}'")
    
    # Invocation de l'agent
    # Les agents attendent une liste de messages. Pour une nouvelle tâche, c'est souvent un HumanMessage.
    try:
        response_planner = planner_agent_executor.invoke({
            "messages": [HumanMessage(content=user_query_plan)]
        })
        research_plan = response_planner.get("output")
        
        print("\n--- Plan de Recherche Généré ---")
        if research_plan:
            print(research_plan)
        else:
            print("L'agent planificateur n'a pas retourné de plan (vérifiez les logs).")
            print(f"Réponse complète de l'agent : {response_planner}")
            
    except Exception as e:
        logger.error(f"Erreur lors de l'invocation du ResearchPlannerAgent: {e}", exc_info=True)
        print(f"Erreur lors du test du ResearchPlannerAgent: {e}")
else:
    logger.warning("Clé API OpenAI manquante, test du ResearchPlannerAgent sauté.")

### 2. Test du `ArxivSearchAgent` (avec `arxiv_search_tool`)

Cet agent est spécialisé dans la recherche d'articles sur ArXiv. Il utilise `arxiv_search_tool`.

In [ ]:
if settings.OPENAI_API_KEY:
    logger.info("\n--- Test du ArxivSearchAgent ---")
    arxiv_agent_executor = create_arxiv_search_agent()
    
    # Tâche de recherche pour l'agent (pourrait être une directive issue d'un plan)
    search_task = "Find 2 recent papers (last 6 months) on 'explainable reinforcement learning in robotics' sorted by submission date."
    # L'agent doit comprendre cette tâche et formuler la bonne requête pour son outil.
    logger.info(f"Tâche pour l'agent de recherche ArXiv : '{search_task}'")
    
    try:
        response_arxiv_search = arxiv_agent_executor.invoke({
            "messages": [HumanMessage(content=search_task)]
        })
        
        # La sortie de l'agent devrait être le résultat de l'appel à l'outil,
        # ou un message indiquant qu'il a utilisé l'outil.
        # Si l'agent est bien conçu, son "output" final devrait être les résultats de l'outil.
        arxiv_results_output = response_arxiv_search.get("output")
        
        print("\n--- Résultats de la Recherche ArXiv (via Agent) ---")
        if arxiv_results_output:
            # La sortie de arxiv_search_tool est une liste de dictionnaires (ou un dict d'erreur)
            # Essayons de l'afficher de manière lisible
            try:
                # Si la sortie est une chaîne JSON, la parser. Sinon, l'afficher telle quelle.
                # L'outil retourne une liste de dicts, mais l'agent LLM pourrait la wrapper en chaîne.
                if isinstance(arxiv_results_output, str):
                    try:
                        parsed_output = json.loads(arxiv_results_output.replace("'", "\"")) # Tentative de correction des apostrophes
                        print(json.dumps(parsed_output, indent=2))
                    except json.JSONDecodeError:
                        print(arxiv_results_output) # Afficher comme chaîne si ce n'est pas du JSON valide
                else: # Supposons que c'est déjà la liste de dicts
                     print(json.dumps(arxiv_results_output, indent=2))
            except Exception as e_print:
                print(f"Erreur lors de l'affichage des résultats : {e_print}")
                print("Sortie brute de l'agent :")
                print(arxiv_results_output)
        else:
            print("L'agent de recherche ArXiv n'a pas retourné de sortie (vérifiez les logs).")
            print(f"Réponse complète de l'agent : {response_arxiv_search}")

        # Pour voir les étapes intermédiaires (appels d'outils) si verbose=True est actif pour l'AgentExecutor
        # et si l'AgentExecutor retourne `intermediate_steps`.
        # Les agents créés avec `create_openai_tools_agent` mettent les `tool_calls` et `tool_messages`
        # dans la clé "messages" de la sortie de `invoke` si on les gère dans un cycle de LangGraph.
        # En invocation directe, `intermediate_steps` peut être disponible.
        if "intermediate_steps" in response_arxiv_search:
            print("\nÉtapes intermédiaires de l'agent ArXiv:")
            for step in response_arxiv_search["intermediate_steps"]:
                tool_call = step[0] # AgentAction (ou equivalent pour tool_calls)
                tool_result = step[1] # Observation (résultat de l'outil)
                print(f"  Appel Outil: {tool_call.tool} avec input {tool_call.tool_input}")
                print(f"  Résultat Outil: {str(tool_result)[:300]}...")
                
    except Exception as e:
        logger.error(f"Erreur lors de l'invocation du ArxivSearchAgent: {e}", exc_info=True)
        print(f"Erreur lors du test du ArxivSearchAgent: {e}")
else:
    logger.warning("Clé API OpenAI manquante, test du ArxivSearchAgent sauté.")

### 2b. Test Direct de `arxiv_search_tool` (Optionnel)

Pour mieux comprendre ce que l'outil `arxiv_search_tool` retourne, nous pouvons l'appeler directement.

In [ ]:
logger.info("\n--- Test Direct de arxiv_search_tool ---")
try:
    direct_tool_results = arxiv_search_tool.invoke({
        "query": "transformer models for robot control", 
        "max_results": 1,
        "sort_by": "submittedDate"
    })
    print("\nRésultats Directs de arxiv_search_tool:")
    print(json.dumps(direct_tool_results, indent=2))
except Exception as e:
    logger.error(f"Erreur lors de l'appel direct à arxiv_search_tool: {e}", exc_info=True)
    print(f"Erreur lors de l'appel direct à arxiv_search_tool: {e}")

### 3. Test du `DocumentAnalysisAgent` (avec `knowledge_base_retrieval_tool`)

Cet agent analyse les documents récupérés de notre base de connaissances (MongoDB). Son bon fonctionnement dépend de la présence de données pertinentes dans la collection (par exemple, `{COLLECTION_NAME_FOR_RAG_TEST}`) et de la fonctionnalité du `RetrievalEngine`.

In [ ]:
if settings.OPENAI_API_KEY and settings.MONGO_URI:
    logger.info("\n--- Test du DocumentAnalysisAgent ---")
    doc_analysis_agent_executor = create_document_analysis_agent()
    
    # Tâche d'analyse pour l'agent.
    # Supposons que nous ayons ingéré des articles sur "explainable artificial intelligence for robotics".
    analysis_task = "Based on the knowledge base, what are some common techniques for achieving explainability in RL agents used in robotics? Cite any relevant ArXiv IDs if found."
    logger.info(f"Tâche pour l'agent d'analyse de documents : '{analysis_task}'")
    
    try:
        response_doc_analysis = doc_analysis_agent_executor.invoke({
            "messages": [HumanMessage(content=analysis_task)]
        })
        analysis_output = response_doc_analysis.get("output")
        
        print("\n--- Résultats de l'Analyse de Documents (via Agent) ---")
        if analysis_output:
            print(analysis_output)
        else:
            print("L'agent d'analyse de documents n'a pas retourné de sortie (vérifiez les logs).")
            print(f"Réponse complète de l'agent : {response_doc_analysis}")

        if "intermediate_steps" in response_doc_analysis:
            print("\nÉtapes intermédiaires de l'agent d'Analyse:")
            for step in response_doc_analysis["intermediate_steps"]:
                tool_call, tool_result = step
                print(f"  Appel Outil: {tool_call.tool} avec input {tool_call.tool_input}")
                print(f"  Résultat Outil (extrait): {str(tool_result)[:300]}...")

    except Exception as e:
        logger.error(f"Erreur lors de l'invocation du DocumentAnalysisAgent: {e}", exc_info=True)
        print(f"Erreur lors du test du DocumentAnalysisAgent: {e}. Assurez-vous que RetrievalEngine peut s'initialiser et que la base de données est peuplée et accessible.")
else:
    logger.warning("Clé API OpenAI ou MONGO_URI manquante, test du DocumentAnalysisAgent sauté.")

### 4. Test du `SynthesisAgent`

Cet agent prend des informations analysées (que nous allons simuler ici) et produit une synthèse structurée. Il n'utilise pas d'outils de récupération.

In [ ]:
if settings.OPENAI_API_KEY:
    logger.info("\n--- Test du SynthesisAgent ---")
    synthesis_agent_executor = create_synthesis_agent()
    
    # Préparer un contexte simulé (ce que les agents précédents auraient pu fournir)
    simulated_context_for_synthesis = """
    User Query: What are key considerations for sim-to-real transfer in robotic reinforcement learning?

    Information from Document Analysis:
    - Chunk from ArXiv ID 123.4567: Sim-to-real transfer often suffers from domain randomization issues. Techniques like domain adaptation and system identification are crucial. Physical parameters like friction and sensor noise are hard to model accurately.
    - Chunk from ArXiv ID 789.0123: Using realistic simulators and adding noise during training can improve transfer. Photorealistic rendering helps vision-based policies. Policy distillation from an ensemble of simulation-trained agents is a promising approach.
    - ArXiv Search found paper 'Recent Advances in Sim-to-Real for Robotics' (ArXiv:2401.0001), summary: This paper reviews state-of-the-art methods, highlighting the importance of robust learning algorithms and accurate dynamics modeling.
    """
    
    synthesis_task_message = HumanMessage(content=f"Based on the provided information below, write a concise summary report on key considerations for sim-to-real transfer in robotic reinforcement learning.\n\nProvided Information:\n{simulated_context_for_synthesis}")
    logger.info(f"Tâche pour l'agent de synthèse (basée sur contexte simulé).")

    try:
        response_synthesis = synthesis_agent_executor.invoke({
            "messages": [synthesis_task_message] # L'agent doit extraire le contexte et la tâche de ce message
        })
        synthesized_output = response_synthesis.get("output")
        
        print("\n--- Sortie de Synthèse (via Agent) ---")
        if synthesized_output:
            print(synthesized_output)
        else:
            print("L'agent de synthèse n'a pas retourné de sortie (vérifiez les logs).")
            print(f"Réponse complète de l'agent : {response_synthesis}")
            
    except Exception as e:
        logger.error(f"Erreur lors de l'invocation du SynthesisAgent: {e}", exc_info=True)
        print(f"Erreur lors du test du SynthesisAgent: {e}")
else:
    logger.warning("Clé API OpenAI manquante, test du SynthesisAgent sauté.")

## Conclusion des Tests d'Agents Individuels

Ce notebook a permis de tester chaque agent de manière isolée pour vérifier son comportement de base et son interaction avec les outils.
Ces tests unitaires sont importants avant d'orchestrer ces agents dans un workflow LangGraph plus complexe (ce que nous ferons dans le notebook suivant).